In [8]:
import pandas as pd
from collections import Counter, defaultdict
import tensorflow as tf
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:

train = pd.read_csv('/home/sanala/Juputer try/PET/data/eng/train.tsv', sep="\t")
dev = pd.read_csv('/home/sanala/Juputer try/PET/data/eng/dev.tsv', sep="\t")
test = pd.read_csv('/home/sanala/Juputer try/PET/data/eng/test.tsv', sep="\t")

train.info()
train.head(10)
dev.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3253 entries, 0 to 3252
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  3253 non-null   object
 1   headline  3253 non-null   object
 2   text      3253 non-null   object
 3   url       3253 non-null   object
dtypes: object(4)
memory usage: 101.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466 entries, 0 to 465
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  466 non-null    object
 1   headline  466 non-null    object
 2   text      466 non-null    object
 3   url       466 non-null    object
dtypes: object(4)
memory usage: 14.7+ KB


In [10]:
#There are repeated examples in the training dataset
print('Unique comments: ', train.iloc[:,1].nunique()==train.shape[0] )
print('Null values: ', train.isnull().values.any())
     

Unique comments:  False
Null values:  False


In [11]:
dev['category'].value_counts()

sports           100
politics          81
business          78
health            74
entertainment     74
technology        59
Name: category, dtype: int64

In [12]:
#Usually the headlines are sort but outliers can appear.
print('average sentence length: ', train.iloc[:,1].str.split().str.len().mean())
print('stdev sentence length: ', train.iloc[:,1].str.split().str.len().std())
print('max sentence length: ', train.iloc[:,1].str.split().str.len().max())

average sentence length:  9.515831540116816
stdev sentence length:  2.017789477997262
max sentence length:  19


In [13]:
train[train.iloc[:,1].duplicated(keep=False).values]

category                                           headline  \
18      sports  Young Sports Personality of the Year 2022: Gym...   
121   politics  Just Stop Oil: Greens co-leader Carla Denyer c...   
921   politics          What's happening in Parliament next week?   
956   politics          What's happening in Parliament next week?   
1122  politics          What's happening in Parliament next week?   
1632  politics          What's happening in Parliament next week?   
1706  politics          What's happening in Parliament next week?   
1767  politics          What's happening in Parliament next week?   
2163  business  'My energy company is refusing to connect my n...   
2241  politics  Just Stop Oil: Greens co-leader Carla Denyer c...   
2831  business  'My energy company is refusing to connect my n...   
3197    sports  Young Sports Personality of the Year 2022: Gym...   

                                                   text  \
18    British world champion gymnast Jessica Gadirov...   
121   Carla Denyer says not all of Just Stop Oil's p...   
921   Boring is the new exciting. Painstaking is the...   
956   It's going to be a pretty weird week in Parlia...   
1122  Parliament is not winding down to Christmas ju...   
1632  It's a week of heavy-duty legislating - with a...   
1706  week will be dominated by scrutiny of the Chan...   
1767  week's drama is backloaded to Thursday, when C...   
2163  Energy company refusing to connect my new home...   
2241  -leader of the Green Party has criticised the ...   
2831  For Pauline Sinclair, building a new home has ...   
3197  British world champion gymnast Jessica Gadirov...   

                                                    url  
18                /sport/av/sports-personality/64059161  
121                          /news/uk-politics-63598138  
921                          /news/uk-politics-63512338  
956                          /news/uk-politics-63347338  
1122                         /news/uk-politics-63921449  
1632                         /news/uk-politics-63838203  
1706                         /news/uk-politics-63676732  
1767                         /news/uk-politics-63596885  
2163  /news/uk-scotland-north-east-orkney-shetland-6...  
2241                         /news/uk-politics-63603255  
2831  /news/uk-scotland-north-east-orkney-shetland-6...  
3197                 /sport/sports-personality/63976879

In [14]:
train_nodup=train.drop_duplicates(subset=['headline'], keep='first')


In [15]:
train_nodup

category                                           headline  \
0            sports  'Marcus Rashford impressing but Manchester Uni...   
1          business  Felixstowe port strike: Union warns of further...   
2        technology       NHS App to offer video consultations by 2024   
3          politics      Gavin Williamson's resignation letter in full   
4     entertainment  Anfield's Gaumont: Art Deco cinema reopens aft...   
...             ...                                                ...   
3248         health  South West Acute Hospital: Over 1,000 attend d...   
3249         health  SWAH surgical applicants did not meet criteria...   
3250         health  The truth about 'medbeds' - a miracle cure tha...   
3251  entertainment  John Hume: Portrait of Nobel laureate unveiled...   
3252     technology  Optus: How a massive data breach has exposed A...   

                                                   text  \
0     Marcus Rashford is fit, firing and having fun....   
1     Dockers at the UK's busiest container port say...   
2     NHS App will soon be updated with features to ...   
3     Gavin Williamson has resigned as Cabinet Offic...   
4     An Art Deco cinema which last welcomed audienc...   
...                                                 ...   
3248  More than 1,000 people have taken part in a de...   
3249  No applicants for vacant consultant general su...   
3250  Strange corners of the internet are awash with...   
3251  An official portrait of the late Nobel Peace P...   
3252  Last week, Australian telecommunications giant...   

                                       url  
0                 /sport/football/64106065  
1        /news/uk-england-suffolk-62713971  
2                    /news/health-61964047  
3               /news/uk-politics-63563100  
4     /news/uk-england-merseyside-63838622  
...                                    ...  
3248    /news/uk-northern-ireland-63840613  
3249    /news/uk-northern-ireland-63949938  
3250         /news/blogs-trending-64070190  
3251    /news/uk-northern-ireland-63537378  
3252        /news/world-australia-63056838  

[3245 rows x 4 columns]

In [16]:
dev[dev.iloc[:,1].duplicated(keep=False).values]

Empty DataFrame
Columns: [category, headline, text, url]
Index: []

In [17]:
dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
 

In [18]:
dev_nodup

category                                           headline  \
0         business         House prices: What happens when they fall?   
1         business  Is it right to raise interest rates in a reces...   
2         business  Pound in biggest monthly fall against the doll...   
3       technology  Meta settles Cambridge Analytica scandal case ...   
4         business  Thai army boycotts e-commerce giant Lazada ove...   
..             ...                                                ...   
461  entertainment   K-pop fans in Birmingham find sense of community   
462       politics  What can we expect from Rishi Sunak as prime m...   
463         sports  Ironman World Championship 70.3: Emma Pallant-...   
464     technology        Google and Microsoft hit by slowing economy   
465  entertainment  LadBaby and Martin Lewis rework Band Aid's Do ...   

                                                  text  \
0    Annual house price growth has slowed, and the ...   
1    Bank of England is like the organiser of a bon...   
2    Worries over the prospects for the UK economy ...   
3    Facebook owner Meta has agreed to pay $725m (£...   
4    's army has boycotted online retailer Lazada o...   
..                                                 ...   
461  Fans of K-pop say they are finding a sense of ...   
462  BBC finds itself criticised now and again for ...   
463  Great Britain's Emma Pallant-Browne took bronz...   
464  Sales at the tech giants Alphabet and Microsof...   
465  YouTube stars LadBaby have unveiled plans to s...   

                                      url  
0               /news/explainers-63147101  
1                 /news/business-63498306  
2                 /news/business-62750606  
3               /news/technology-64075067  
4                 /news/business-61389117  
..                                    ...  
461  /news/uk-england-birmingham-64046847  
462            /news/uk-politics-63381896  
463             /sport/triathlon/63435145  
464               /news/business-63394516  
465     /news/entertainment-arts-63917114  

[466 rows x 4 columns]

In [19]:
#We take X examples from each label, and the rest become the unlabeled data-set.
classes = train['category'].unique()
samples_per_class=5 #5 10 20 50
# create an empty list to store the randomly selected samples
random_samples = []

# iterate through each class
for class_name in classes:
    # select all rows with the current class
    class_rows = train_nodup[train_nodup['category'] == class_name]
    # randomly select X rows from the class_rows DataFrame
    random_class_samples = class_rows.sample(samples_per_class)
    # add the randomly selected samples to the list
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_train=random_samples_df.reset_index(drop=True)
unlabeled_train = train.drop(new_train.index)
#new_train
#unlabeled_train

In [20]:
#new_train.columns = [0, 1, 2,3]
#unlabeled_train.columns = [0, 1, 2,3]

In [21]:
random_samples = []
samples_per_class=5
for class_name in classes:
    class_rows = dev_nodup[dev_nodup['category'] == class_name]
    random_class_samples = class_rows.sample(samples_per_class)
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_dev=random_samples_df.reset_index(drop=True)
#new_dev.columns = [0, 1, 2,3]

In [22]:
unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
unlabeled_train.reset_index(drop=True)

unlabeled       category  \
0     unlabeled       business   
1     unlabeled         sports   
2     unlabeled  entertainment   
3     unlabeled  entertainment   
4     unlabeled       business   
...         ...            ...   
3218  unlabeled         health   
3219  unlabeled         health   
3220  unlabeled         health   
3221  unlabeled  entertainment   
3222  unlabeled     technology   

                                               headline  \
0     National Grid raises discounts for off-peak el...   
1     Watch: Bollywood and the IPL from The Great In...   
2     Letter from Africa: The Afro-punk band taking ...   
3     British Nigerian artist Dylema paints African ...   
4     Truss still needs to win over both markets and...   
...                                                 ...   
3218  South West Acute Hospital: Over 1,000 attend d...   
3219  SWAH surgical applicants did not meet criteria...   
3220  The truth about 'medbeds' - a miracle cure tha...   
3221  John Hume: Portrait of Nobel laureate unveiled...   
3222  Optus: How a massive data breach has exposed A...   

                                                   text  \
0     ue of discounts on electricity bills for house...   
1     Indian Premier League has captured the hearts ...   
2     In our series of letters from African journali...   
3     rtist known as Dylema was diagnosed with polyc...   
4     rime minister's acknowledgement that the mini-...   
...                                                 ...   
3218  More than 1,000 people have taken part in a de...   
3219  No applicants for vacant consultant general su...   
3220  Strange corners of the internet are awash with...   
3221  An official portrait of the late Nobel Peace P...   
3222  Last week, Australian telecommunications giant...   

                                     url  
0                /news/business-63329233  
1             /sport/av/cricket/63724650  
2            /news/world-africa-63034861  
3            /news/world-africa-63434248  
4                /news/business-63108297  
...                                  ...  
3218  /news/uk-northern-ireland-63840613  
3219  /news/uk-northern-ireland-63949938  
3220       /news/blogs-trending-64070190  
3221  /news/uk-northern-ireland-63537378  
3222      /news/world-australia-63056838  

[3223 rows x 5 columns]

In [23]:
unlabeled_train.drop(columns='category', inplace=True)
unlabeled_train

unlabeled                                           headline  \
30    unlabeled  National Grid raises discounts for off-peak el...   
31    unlabeled  Watch: Bollywood and the IPL from The Great In...   
32    unlabeled  Letter from Africa: The Afro-punk band taking ...   
33    unlabeled  British Nigerian artist Dylema paints African ...   
34    unlabeled  Truss still needs to win over both markets and...   
...         ...                                                ...   
3248  unlabeled  South West Acute Hospital: Over 1,000 attend d...   
3249  unlabeled  SWAH surgical applicants did not meet criteria...   
3250  unlabeled  The truth about 'medbeds' - a miracle cure tha...   
3251  unlabeled  John Hume: Portrait of Nobel laureate unveiled...   
3252  unlabeled  Optus: How a massive data breach has exposed A...   

                                                   text  \
30    ue of discounts on electricity bills for house...   
31    Indian Premier League has captured the hearts ...   
32    In our series of letters from African journali...   
33    rtist known as Dylema was diagnosed with polyc...   
34    rime minister's acknowledgement that the mini-...   
...                                                 ...   
3248  More than 1,000 people have taken part in a de...   
3249  No applicants for vacant consultant general su...   
3250  Strange corners of the internet are awash with...   
3251  An official portrait of the late Nobel Peace P...   
3252  Last week, Australian telecommunications giant...   

                                     url  
30               /news/business-63329233  
31            /sport/av/cricket/63724650  
32           /news/world-africa-63034861  
33           /news/world-africa-63434248  
34               /news/business-63108297  
...                                  ...  
3248  /news/uk-northern-ireland-63840613  
3249  /news/uk-northern-ireland-63949938  
3250       /news/blogs-trending-64070190  
3251  /news/uk-northern-ireland-63537378  
3252      /news/world-australia-63056838  

[3223 rows x 4 columns]

In [603]:

#We take X examples from each label, and the rest become the unlabeled data-set.
#Instead, we could also make the new training data-set following the proportion of examples per label.
#PVP pattern-verbalizer pair (PVP)

In [604]:
new_train.to_csv("/home/sanala/Juputer try/pet/data-fsl/eng/5sample/train.tsv", sep="\t", index=False  ,header=False)
unlabeled_train.to_csv("/home/sanala/Juputer try/pet/data-fsl/eng/5sample/unlabeled.tsv", sep="\t", index=False,header=False)
new_dev.to_csv("/home/sanala/Juputer try/pet/data-fsl/eng/5sample/dev.tsv", sep="\t", index=False,header=False)

### Train your own PET



In [224]:
%cd /home/sanala/Juputer try/pet/


/home/sanala/Juputer try/pet
